In [ ]:
import tensorflow as tf

import pathlib
from typing import Any,Union

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_dataset_dir(path) -> str:
    return pathlib.Path(path)

In [ ]:
BATCH_SIZE:int  = 64
IMG_HEIGHT:int  = 180
IMG_WIDTH:int   = 180

In [ ]:
import csv

MODEL_NAME = 'model_09.20.2022_20:31:25'
model:tf.keras.Model = tf.keras.models.load_model(MODEL_NAME)

In [ ]:
from glob import glob

def testAndPrint(model_):
    validation_path = pathlib.Path("dataset")
    class_names = list(glob('*',root_dir='dataset'))
    print(class_names)

    isPhotoError = 0
    miss = {}
    for class_name in class_names:
        miss[class_name] = 0

    for class_name in class_names:
        err = 0
        sum_perc = 0
        notdef = 0
        plt.figure(figsize=(100, 100))
        for class_name_2 in class_names:
            miss[class_name_2] = 0
        for path in list(validation_path.glob(f"{class_name}/*.jpg")):

            img = tf.keras.utils.load_img(
                path, target_size=(IMG_HEIGHT, IMG_WIDTH)
            )
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0) # Create a batch

            predictions = model_.predict(img_array,verbose=0)
            score = tf.nn.softmax(predictions[0])

            miss[class_names[np.argmax(score)]] += 1
            #if 100 * np.max(score) < 80:
            #        notdef = notdef + 1
            #else:

            if class_name != class_names[np.argmax(score)]:
                if class_name == 'photo' or class_names[np.argmax(score)]=='photo':
                    isPhotoError +=1
                #print(
                #    "{} {} with a {:.2f} percent confidence but is a {}"
                #    .format(path, class_names[np.argmax(score)], 100 * np.max(score), class_name)
                #)

                err = err + 1
                sum_perc = sum_perc + 100 * np.max(score)
                if err <= 40:
                    plt.subplot(5, 8, err)
                    plt.imshow(img)
                    plt.title(class_names[np.argmax(score)],fontsize=60)
                    plt.axis("off")


        len_class = len(list(validation_path.glob(f"{class_name}/*.jpg")))
        if err > 0:
            stat = ((len_class-err-notdef) / len_class) * 100
            print(f"{class_name} {stat}% || {err} error || {notdef} nodef || {sum_perc/err}")
        else:
            stat = ((len_class-notdef) / len_class) * 100
            print(f"{class_name} {stat}% || {notdef} nodef")
        print(f"{class_name}",miss)
    print(isPhotoError)
    plt.show()


In [ ]:
testAndPrint(model)